In [383]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [384]:
filename = "06_ABox.ttl"

In [385]:
text1 = '''MS1
Provide a list of all Open Access Publications for 2023 for Journal Articles (Gold (APC Driven), Green, Diamond) published by authors affiliated with University of Bologna using the OpenAIRE Graph.
'''

query1 = '''
PREFIX tbox: <https://example.org/skg-if/extention/02/schema/>
PREFIX : <https://example.org/skg-if/extention/02/data/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?ror_id ?org_name ?article_title (GROUP_CONCAT(DISTINCT ?article_doi; separator=", ") AS ?article_dois) ?open_access_type (YEAR(?issued_date) AS ?year)
WHERE {
    
    ?article a tbox:Work ;
        tbox:title ?article_title ;
        tbox:hasIdentifier [ tbox:hasLiteralValue ?article_doi ; tbox:usesIdentifierScheme tbox:doi ] ;
        tbox:isRelatedToRoleInTime [ tbox:withRole tbox:affiliate ; tbox:relatesToOrganization :university-of-bologna ] ;
        tbox:realization [ 
            a ?type ;
            tbox:issued ?issued_date ;
            tbox:holdsStatusInTime [ tbox:withStatus ?open_access_type ] 
        ] .
    
    :university-of-bologna tbox:name ?org_name ;
        tbox:hasIdentifier [ tbox:hasLiteralValue ?ror_id ; tbox:usesIdentifierScheme tbox:ror ] .
    
    FILTER (
        ?type = tbox:JournalArticle
        &&
        ?issued_date >= "2023-01-01T00:00:00+00:00"^^xsd:dateTime 
        &&
        ?issued_date <= "2023-12-31T23:59:59+00:00"^^xsd:dateTime
    )

}
GROUP BY ?article
'''

In [386]:
text2 = '''MS2
Provide a list of all datasets identified in OpenAIRE that are affiliated with University of Bologna, in any year between 2014-2024.
'''

query2 = '''
PREFIX tbox: <https://example.org/skg-if/extention/02/schema/>
PREFIX : <https://example.org/skg-if/extention/02/data/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?ror_id ?org_name ?dataset_title (GROUP_CONCAT(DISTINCT ?dataset_doi; separator=", ") AS ?dataset_dois) (YEAR(?issued_date) AS ?year)
WHERE {
    
    ?dataset a tbox:Dataset ;
        tbox:title ?dataset_title ;
        tbox:hasIdentifier [ tbox:hasLiteralValue ?dataset_doi ; tbox:usesIdentifierScheme tbox:doi ] ;
        tbox:isRelatedToRoleInTime [ tbox:withRole tbox:affiliate ; tbox:relatesToOrganization :university-of-bologna ] ;
        tbox:realization [ tbox:issued ?issued_date ] .
    
    :university-of-bologna tbox:name ?org_name ;
        tbox:hasIdentifier [ tbox:hasLiteralValue ?ror_id ; tbox:usesIdentifierScheme tbox:ror ] .
    
    FILTER (
        ?issued_date >= "2014-01-01T00:00:00+00:00"^^xsd:dateTime 
        &&
        ?issued_date <= "2024-12-31T23:59:59+00:00"^^xsd:dateTime
    )

}
GROUP BY ?dataset

'''

In [387]:
text3 = '''MS3
Provide a list of all Preprints identified in OpenAIRE that are affiliated with University of Bologna, in any year between 2014-2024.
'''

query3 = '''
PREFIX tbox: <https://example.org/skg-if/extention/02/schema/>
PREFIX : <https://example.org/skg-if/extention/02/data/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?ror_id ?org_name ?article_title (GROUP_CONCAT(DISTINCT ?article_doi; separator=", ") AS ?article_dois) ?type_label (YEAR(?issued_date) AS ?year) ?repository_name
WHERE {

    ?article a tbox:Work ;
        tbox:title ?article_title ;
        tbox:hasIdentifier [ tbox:hasLiteralValue ?article_doi ; tbox:usesIdentifierScheme tbox:doi ] ;
        tbox:isRelatedToRoleInTime [ tbox:withRole tbox:affiliate ; tbox:relatesToOrganization :university-of-bologna ] ;
        tbox:realization [ 
            a ?type ;
            tbox:issued ?issued_date ;
            tbox:accessService [
                tbox:name ?repository_name
            ]
        ] .

    :university-of-bologna tbox:name ?org_name ;
        tbox:hasIdentifier [ tbox:hasLiteralValue ?ror_id ; tbox:usesIdentifierScheme tbox:ror ] .
    
    ?type tbox:prefLabel ?type_label .
    
    FILTER (
        ?type = tbox:Preprint
        &&
        ?issued_date >= "2014-01-01T00:00:00+00:00"^^xsd:dateTime 
        &&
        ?issued_date <= "2024-12-31T23:59:59+00:00"^^xsd:dateTime
    )
    
}
GROUP BY ?article

'''

In [388]:
queries = [(text1, query1),
           (text2, query2),
           (text3, query3)]

g = rdflib.Dataset()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))
    print("\n")

MS1
Provide a list of all Open Access Publications for 2023 for Journal Articles (Gold (APC Driven), Green, Diamond) published by authors affiliated with University of Bologna using the OpenAIRE Graph.

+---------------------------+-----------------------+--------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------+------------------------------------------------------------------+--------+
| ror_id                    | org_name              | article_title                                                                        | article_dois                                                                          | open_access_type                                                 |   year |
|---------------------------+-----------------------+--------------------------------------------------------------------------------------+-----------------------------------------------